In [2]:
import numpy as np
import scipy.misc
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from keras import backend as K
import cv2
import os

In [2]:
np.random.seed(1)
names = ["Apple Scab", "Apple Black Rot", "Apple Cedar Apple Rust", "Apple Healthy", "Blueberry Healthy",
         "Cherry Powdery Mildew", "Cherry Healthy", "Corn Cercospora Gray Leaf Spot", "Corn Common Rust",
         "Corn Northern Leaf Blight", "Corn Healthy", "Grape Black Rot", "Grape Esca", "Grape Leaf Blight",
         "Grape Healthy", "Orange Haunglongbing", "Peach Bacterial Spot", "Peach Healthy", "Bell Pepper Bacterial Spot",
         "Bell Pepper Healthy", "Potato Early Blight", "Potato Late Blight", "Potato Healthy", "Raspberry Healthy",
         "Soybean Healthy", "Squash Powdery Mildew", "Strawberry Leaf Scorch", "Strawberry Healthy",
         "Tomato Bacterial Spot", "Tomato Early Blight", "Tomato Late Blight", "Tomato Leaf Mold",
         "Tomato Septoria Leaf Spot", "Tomato Spider Mites", "Tomato Target Spot", "Tomato Yellow Leaf Curl Virus",
         "Tomato Mosaic Virus", "Tomato Healthy"]


In [2]:
base = "E:\\PycharmProjects\\Chameleon\\data\\color\\"


In [2]:
images = []
labels = []
i = 0
for filename1 in os.listdir(base):
    count = 0
    for filename in os.listdir(base + filename1):
        img1 = cv2.imread(base + filename1 + "\\" + filename)
        img1 = img1.transpose((-1, 0, 1))
        if img1 is not None:
            labels.append(i)
            images.append(img1)
            count = count + 1
        if count == 256:
            break
    if i % 5 == 0:
        print(i)
    i = i + 1


In [16]:
X_train = np.array(images)
y_train = np.array(labels)
y_train = y_train.reshape(np.shape(X_train)[0], 1)
p = np.random.permutation(np.shape(X_train)[0])
X_train = X_train[p]
y_train = y_train[p]
y_train = np_utils.to_categorical(y_train)

In [2]:
print(np.shape(X_train))
print(y_train[0])

In [2]:
num_classes = 38
K.set_image_dim_ordering('th')


def train():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(3, 256, 256), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), input_shape=(3, 256, 256)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model


trained_model = train()


In [2]:
trained_model.fit(
    X_train, y_train,
    epochs=50, batch_size=64)
trained_model.save_weights('first_try.h5')


In [2]:
img = cv2.imread("E:\\PycharmProjects\\Chameleon\\data\\test_set\\t1.JPG")
img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_CUBIC)
plt.imshow(img)
plt.show()
img = img.transpose((-1, 0, 1))
X_test = [img, img]

In [3]:
y_test = trained_model.predict_classes(np.array(X_test))
print(y_test)

In [151]:
im = []
for i in range(3, 4):
    for filename in os.listdir(base + str(i)):
        img1 = cv2.imread(base + str(i) + "\\" + filename)
        img1 = cv2.resize(img1, (256, 256), interpolation=cv2.INTER_CUBIC)
        if img1 is not None:
            labels.append(i)
            im.append(img1)
im = np.reshape(im, (np.shape(im)[0], 3, 256, 256))
y_test = model.predict_classes(im)
count = len(im)
for i in range(len(y_test)):
    if y_test[i] == 3:
        count = count - 1
print(count)
print(len(y_test))


713
713


In [ ]:
import requests

p = np.random.permutation(38)
print(names[p[0]])
url = "https://nagarro.herokuapp.com/" + names[p[0]]
response = requests.post(url, headers={"Content-Type": "application/json"})
about = response.json()['value']['about']
print("About")
print(about)
cure = response.json()['value']['cure']
print("Treatment")
print(cure)


Corn Northern Leaf Blight


About
Northern corn leaf blight is a foliar disease of corn caused by Exserohilum turcicum, the anamorph of the ascomycete Setosphaeria turcica. With its characteristic cigar-shaped lesions, this disease can cause significant yield loss in susceptible corn hybrids.
Treatment
Growers can manage NLB by selecting resistant hybrids, reducing previous corn residue, timely planting, and in high-risk fields with recent NLB outbreaks, application of fungicides. To determine level of risk, growers are encouraged to routinely scout fields for presence of NLB and other leaf diseases. Scouting is also important to identify any stalk quality problems that often result from leaf diseases.
